# Metropolis Hastings

## Algorithm

The Metropolis-Hastings algorithm is a general term for a family of Markov chain simulation methods that are useful for sampling from Bayesian posterior distributions. MHA, can be considered as a generalisation of the Metropolis algorithm for **Non Symmetric Jump Distributions**.

Both Gibbs sampling and Metropolic Algorithm are special cases of Metropolis Hastings Algorithm.

The Metropolis-Hastings algorithm generalizes the basic Metropolis algorithm presented above in two ways. 
1. The jumping rules $J_t$ need no longer be symmetric; that is, there is no requirement that 
$J_t(\theta_a|\theta_b) ≡ J_t(\theta_b|\theta_a)$. 
2. To correct for the asymmetry in the jumping rule, the ratio r for acceptance probability is replaced by a ratio of ratios:

$$
r = \frac{p(\theta^{*}|y)/J_t(\theta^{*}|\theta^{t-1})}{p(\theta^{t-1}|y)/J_t(\theta^{t-1}|\theta^{*})}
$$

This leads to a stationary distribution for the target pdf
